In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv('augmented_user_behaviour_3000_updated.csv')

In [3]:
df.head()

,tap_pressure,swipe_speed,gesture_type,typing_speed,inter_key_delay,grip_x,grip_y,grip_z,tilt_angle_mean,screen_sequence,...,geolocation_cluster,wifi_network_hash,label,user_id,trait_categories,device_tilt_variation,fast_typing_burst,gps_drift,transaction_amount,transaction_date
0,0.428514,1215.821203,scroll,3.296831,145.180401,0.850983,0.757304,0.605133,28.598586,"transfer,bill,offers",...,travel,-3.040000e+17,0,user_045,"Contextual Pattern, Device Handling, Navigatio...",6.495890,0,22.247370,4898.65,07-04-2025 04:53
1,0.285105,601.456372,swipe,0.813050,47.492616,-3.448888,-2.951799,-0.815461,0.430524,"offers,home,bill",...,unknown,5.100000e+18,1,user_012,"Contextual Pattern, Device Handling, Navigatio...",4.162963,0,116.903749,2596.02,23-08-2024 03:51
2,0.541403,1447.570971,tap,5.941504,138.055740,-0.611922,1.809515,-1.474237,20.762958,"home,transfer,bill",...,travel,-4.950000e+18,0,user_014,"Contextual Pattern, Device Handling, Navigatio...",6.671911,1,23.489516,5696.96,15-05-2025 04:56
3,0.568683,1472.510297,scroll,4.452875,198.368348,1.314914,1.639965,0.742127,30.377168,"transfer,bill,home",...,travel,-3.040000e+17,0,user_007,"Contextual Pattern, Device Handling, Navigatio...",8.623862,0,16.070070,13670.97,11-01-2025 02:26
4,0.499482,1465.225236,tap,4.838592,128.846684,-1.349605,0.019995,-0.399827,46.886915,"offers,bill,transfer",...,home,2.850000e+18,0,user_033,"Contextual Pattern, Device Handling, Navigatio...",10.304295,0,15.186337,2358.65,01-09-2024 09:05


In [4]:
ndf=df.drop(columns=['trait_categories','user_id'])
ndf.head()
ndf.shape
test_df=pd.read_csv('imbalanced_user_behaviour.csv')
test_df.head()
y_test_im=test_df['label']
y_test_im.shape
X_test_im=test_df.drop(columns=['label','user_id','trait_categories'])
X_test_im.head()

,tap_pressure,swipe_speed,gesture_type,typing_speed,inter_key_delay,grip_x,grip_y,grip_z,tilt_angle_mean,screen_sequence,dwell_time_avg,geolocation_cluster,wifi_network_hash,device_tilt_variation,fast_typing_burst,gps_drift,transaction_amount,transaction_date
0,0.462015,1145.786289,tap,3.971542,95.065654,-1.228444,-2.075588,-0.086070,29.248398,"home,profile,transfer",3.152831,travel,-303768024939284139,9.173079,0,18.147460,4513.00,2025-05-27 09:41:08
1,0.483327,1803.990401,tap,1.364200,172.793617,-0.839528,1.372271,0.142044,26.635473,"bill,transfer,offers",4.732641,unknown,2853320367896827313,2.965487,0,19.386455,1410.83,2025-01-03 16:21:04
2,0.498596,1567.047737,swipe,2.552508,131.214299,0.015379,0.955096,-1.032869,25.707750,"profile,home,offers",4.025024,office,-4946583138965503492,13.011113,0,18.601199,3802.02,2025-01-25 04:51:52
3,0.541885,1828.454456,tap,4.504338,129.121908,1.992366,1.001403,-0.457357,26.123368,"bill,offers,home",4.148015,travel,-4946583138965503492,6.015742,0,29.148101,714.39,2025-01-03 15:19:20
4,0.501736,1106.299157,tap,4.154976,174.323959,-0.993863,0.420192,-0.856498,30.488352,"offers,transfer,bill",2.853193,unknown,2853320367896827313,6.254326,0,24.511386,1232.51,2025-03-11 05:51:59


In [5]:
# Fixing date parsing by specifying correct format and using dayfirst=True
#ndf["transaction_date"] = pd.to_datetime(ndf["transaction_date"], format="%d-%m-%Y %H:%M", dayfirst=True)

# Extract time-based features
#ndf["hour"] = ndf["transaction_date"].dt.hour
#ndf["day_of_week"] = ndf["transaction_date"].dt.dayofweek
#ndf["is_weekend"] = ndf["day_of_week"].apply(lambda x: 1 if x >= 5 else 0)
#ndf["is_night"] = ndf["hour"].apply(lambda h: 1 if h <= 5 else 0)

test_df["transaction_date"] = pd.to_datetime(test_df["transaction_date"], format="%Y-%m-%d %H:%M:%S")

# Extract time-based features
test_df["hour"] = test_df["transaction_date"].dt.hour
test_df["day_of_week"] = test_df["transaction_date"].dt.dayofweek
test_df["is_weekend"] = test_df["day_of_week"].apply(lambda x: 1 if x >= 5 else 0)
test_df["is_night"] = test_df["hour"].apply(lambda h: 1 if h <= 5 else 0)







In [6]:
x=ndf.drop(columns=['label','transaction_date'])
y=ndf['label']

In [7]:
x.head()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tap_pressure           3000 non-null   float64
 1   swipe_speed            3000 non-null   float64
 2   gesture_type           3000 non-null   object 
 3   typing_speed           3000 non-null   float64
 4   inter_key_delay        3000 non-null   float64
 5   grip_x                 3000 non-null   float64
 6   grip_y                 3000 non-null   float64
 7   grip_z                 3000 non-null   float64
 8   tilt_angle_mean        3000 non-null   float64
 9   screen_sequence        3000 non-null   object 
 10  dwell_time_avg         3000 non-null   float64
 11  geolocation_cluster    3000 non-null   object 
 12  wifi_network_hash      3000 non-null   float64
 13  device_tilt_variation  3000 non-null   float64
 14  fast_typing_burst      3000 non-null   int64  
 15  gps_

In [8]:
numeric_features=x.select_dtypes(include=['int64','float64','int32']).columns.to_list()
categorical_features=x.select_dtypes(include=['object']).columns.to_list()



In [9]:
#numeric_features = ['hour', 'day_of_week', 'is_weekend', 'is_night', ...]  # add your other numeric features here

In [10]:
numeric_features


['tap_pressure',
 'swipe_speed',
 'typing_speed',
 'inter_key_delay',
 'grip_x',
 'grip_y',
 'grip_z',
 'tilt_angle_mean',
 'dwell_time_avg',
 'wifi_network_hash',
 'device_tilt_variation',
 'fast_typing_burst',
 'gps_drift',
 'transaction_amount']

In [11]:
# Define transformers
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore",sparse_output=False))
])

# Combine transformations
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])
# Define full pipeline
pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
])

# Making Model using Autoencoder

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_autoencoder(input_dim):
    input_layer = layers.Input(shape=(input_dim,))
    encoded = layers.Dense(64, activation='relu')(input_layer)
    encoded = layers.Dense(32, activation='relu')(encoded)

    decoded = layers.Dense(64, activation='relu')(encoded)
    output = layers.Dense(input_dim)(decoded)

    autoencoder = models.Model(inputs=input_layer, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [13]:
from sklearn.model_selection import train_test_split

# Preprocess data
#X = ndf[numeric_features + categorical_features]
X = test_df[numeric_features + categorical_features]
X_processed = pipeline.fit_transform(X)

# Split data
X_train, X_test = train_test_split(X_processed, test_size=0.2, random_state=42, stratify=y_test_im)

# Build and train autoencoder
autoencoder = build_autoencoder(X_train.shape[1])
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1885 - val_loss: 0.1167
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0993 - val_loss: 0.0505
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0453 - val_loss: 0.0309
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0271 - val_loss: 0.0238
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0211 - val_loss: 0.0198
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0177 - val_loss: 0.0170
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0160 - val_loss: 0.0150
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0149 - val_loss: 0.0141
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0135 - val_loss: 0.0138
Epoch 10/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0131 - val_loss: 0.0131
Epoch 11/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0125 - val_loss: 0.0127
Epoch 12/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [14]:
X_train_2, X_test_2,y_train,y_test=train_test_split(X_processed,y_test_im,test_size=0.2,random_state=42,stratify=y_test_im)
y_train.shape
y_test.shape
X_train.shape

(4000, 82)

In [16]:
import numpy as np

# Predict on test set
reconstructions = autoencoder.predict(X_test)
errors = np.mean(np.square(X_test - reconstructions), axis=1)

# Set dynamic threshold (e.g., 90th percentile of training errors)
#threshold = np.percentile(errors, 90)
# Convert errors to predicted labels
#y_pred = (errors > threshold).astype(int)

from sklearn.metrics import f1_score

# Tune threshold to maximize F1 score
best_f1 = 0
best_thresh = 0
for t in np.linspace(min(errors), max(errors), 200):
    preds = (errors > t).astype(int)
    f1 = f1_score(y_test, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

# Final prediction using best threshold
threshold = best_thresh
y_pred = (errors > threshold).astype(int)
print(f"Best threshold based on F1 score: {threshold:.4f}")



32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Best threshold based on F1 score: 0.0178


In [17]:

import numpy as np
# Get reconstruction error
reconstructions = autoencoder.predict(X_test)
errors = np.mean(np.square(X_test - reconstructions), axis=1)

# Define thresholds
#low_thresh = np.percentile(errors, 70)
#high_thresh = np.percentile(errors, 90)
# 3-level thresholds relative to tuned binary threshold
low_thresh = np.percentile(errors, 60)
high_thresh = threshold  # Use your optimized threshold for "high" risk

# Risk Scoring
def get_risk_label(error):
    if error < low_thresh:
        return 'Low'
    elif error < high_thresh:
        return 'Moderate'
    else:
        return 'High'

risk_labels = [get_risk_label(e) for e in errors]


# Risk Scoring
#def get_risk_label(error):
    #if error < low_thresh:
        #return 'Low'
    #elif error < high_thresh:
       # return 'Moderate'
    #else:
        #return 'High'

risk_labels = [get_risk_label(e) for e in errors]

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [18]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Ground truth labels
y_true = y_test

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Classification report
print(classification_report(y_true, y_pred, target_names=['Normal', 'Anomaly']))

# AUC Score (optional but useful)
roc_auc = roc_auc_score(y_true, errors)
print("ROC-AUC Score:", roc_auc)


Confusion Matrix:
 [[990   0]
 [  1   9]]
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       990
     Anomaly       1.00      0.90      0.95        10

    accuracy                           1.00      1000
   macro avg       1.00      0.95      0.97      1000
weighted avg       1.00      1.00      1.00      1000

ROC-AUC Score: 0.9673737373737374


In [19]:
import joblib
pipeline.fit(X)  # Make sure it's fitted on your full data
joblib.dump(pipeline, "pipeline.joblib")
autoencoder.save("autoencoder_model.keras")